In [79]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## モデルをチェインとして記述する

In [80]:
l1 = L.Linear(4,3)
l2 = L.Linear(3,2)

def my_forward(x):
    h = l1(x)
    return l2(h)

#4要素のベクトルの2個のミニバッチ
data = np.array([[1,2,3,4],
                [4,5,6,7]], dtype=np.float32)
x = Variable(data)

#forward計算で2要素のベクトル×2が出て来る
my_forward(x).data


array([[-1.19160485, -3.2900362 ],
       [-3.01705313, -6.96557999]], dtype=float32)

一応関数で此のようにモデルを書くことはできるが、再利用が難しい

クラスで書くほうが望ましい


In [81]:
class MyProc(object):
    def __init__(self):
        self.l1 = L.Linear(4,3)
        self.l2 = L.Linear(3,2)
    
    def forward(self, x):
        h = self.l1(x)
        return self.l2(h)

クラスで書くとこのようになる
さらに、CPU/GPUサポートや、save/load機能などをサポートするにはChainクラスを継承することで可能となる

In [82]:
class MyChain(Chain):
    def __init__(self):
        super(MyChain, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(4,3)
            self.l2 = L.Linear(3,2)
    
    def forward(self, x):
        h = self.l1(x)
        return self.l2(h)

このように書いた時、このMyChainの中のl1やl2のリンクを、MyChainの子リンクと呼んだりする

さらに、ChainクラスはLinkを継承している。これにより、さらに複雑なチェインがMyChainを子リンクとして持つことも可能

学習可能なレイヤ（Linkなど）をChainへ登録するのに、v1ではChainクラスのコンストラクタやadd_link関数を通していたが、v2では上の用に書くことが推奨される

In [83]:
class MyChain(Chain):
    def __init__(self):
        super(MyChain, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(4,3)
            self.l2 = L.Linear(3,2)
    
    def __call__(self, x):
        h = self.l1(x)
        return self.l2(h)

#インスタンス作成
ch = MyChain()
#__call__の定義により関数のように使うことができる
ch(x).data


array([[  0.46987295,  -5.81955385],
       [  0.9263339 , -13.46533489]], dtype=float32)

クラス内で__call__を定義すると関数のように呼ぶことができる

### ChainList

In [84]:
class MyChain2(ChainList):
    def __init__(self):
        super(MyChain2, self).__init__(
            L.Linear(4,3),
            L.Linear(3,2)
        )
                
    def __call__(self, x):
        h = self[0](x)
        return self[1](h)

ChainListを継承すると任意の数のリンクを便利に扱うことができる

ただしリンク数が固定の場合はChainクラスを継承することが推奨される

### Optimizer

In [85]:
model = MyChain()
optimizer = optimizers.SGD()
optimizer.setup(model)

パラメータの最適化アルゴリズムがoptimizers下に実装されている

optimizer.setup(model)

でmodel上でoptimizerが働くようになる

下記のようにして、勾配計算後、パラメータ更新前に呼び出されるフック関数を設定できる（正則化関数等）


In [86]:
optimizer.add_hook(chainer.optimizer.WeightDecay(0.0005))

### Directly Use Optimizer

Optimizerを直接扱う場合について説明する

直接扱う場合にも2通りの方法がある

まず、下記では「手動で勾配を計算して、update()関数を呼び出す方法」について説明する

In [87]:
#ミニバッチ2個で4次元データを用意
x = np.random.uniform(-1, 1, (2, 4)).astype('f')

#勾配を初期化
model.cleargrads()

'結果は2次元×ミニバッチ2個'
model(Variable(x))

#ミニバッチも含めた結果全ての和をlossとする
loss = F.sum(model(Variable(x)))

#勾配計算
loss.backward()

#MyChainのWを一つ取り出して、勾配計算結果と、パラメータの更新を見てみる
p = model.params()

temp = p.__next__()
temp = p.__next__()

#重み行列の1つとその計算された勾配
'更新前の重みと勾配'
temp 
temp.grad

#optimizerが重みを更新する
optimizer.update()

#更新後　おそらくデフォルトの学習係数は0.01
#W_new = W_old - 0.01 * W.grad ぐらいにだいたいなってることが分かる
'更新後の重み'
temp 
temp.grad

'結果は2次元×ミニバッチ2個'

variable([[ 0.04464273,  0.10187956],
          [-0.19118905, -0.0876995 ]])

'更新前の重みと勾配'

variable W([[ 0.70131022,  0.16021767,  0.15233542],
            [ 0.82547462,  0.14936391, -0.20094587]])

array([[-0.03631453, -0.32036534, -0.45787361],
       [-0.03631453, -0.32036534, -0.45787361]], dtype=float32)

'更新後の重み'

variable W([[ 0.70166987,  0.16342053,  0.1569134 ],
            [ 0.82583362,  0.15256681, -0.19636613]])

array([[-0.03596388, -0.32028523, -0.45779744],
       [-0.0359018 , -0.32029065, -0.45797408]], dtype=float32)

もう1つの方法として、optimizerのupdate関数にloss関数を渡す方法がある

In [88]:
def lossfun(arg1, arg2):
    loss = F.sum(model(arg1-arg2))
    return loss

x1 = np.random.uniform(-1, 1, (2, 4)).astype('f')
x2 = np.random.uniform(-1, 1, (2, 4)).astype('f')

#modelの初期化は自動で行われる
optimizer.update(lossfun, Variable(x1), Variable(x2))


この方法のとき、modelの勾配の初期化は自動で行われる

### Trainer

Training loop…パラメータ更新のため行われる作業の繰り返し

典型的には以下のプロセスからなる

1. 学習用データセットをなめる
1. ミニバッチの前処理
1. NNを使ってForward/Backwardの計算
1. パラメータ更新
1. 現在のパラメータでテスト用データを使って計算
1. 中間結果の表示・ログ

データセット抽象化 … 上記1と2を解決する。主にdatasetモジュールで定義されている

Trainer … 上記3～6を実装する。手順全体はTrainerに実装されている。3と4はUpdaterに定義されており、カスタマイズ可能。5と6はExtensionに実装されており、Training loopに追加の手順を加えることができる。これでTraining loopを自由にカスタマイズできる他、ユーザー定義Extensionも作ることができる。

Trainerに入る前にSerializerについて説明する

### Serializer

Link, Optimizer, Trainerのシリアライズ用のモジュール
npz形式とhdf5形式がある

In [96]:
optimizer.update(lossfun, x1-x2, x2-x1)

params = model.params()
W = params.__next__()
W = params.__next__()
W
W.grad

#ロード
'ロード'
serializers.load_npz('my.model', model)

#重みは保存されたものをロードしている　勾配はもともとmodelに入れられていたのがそのまま残っている
#（ファイルには保存されていない）
params = model.params()
W = params.__next__()
W = params.__next__()
W
'重みはファイルに保存されていない'
W.grad

variable W([[ 0.61994195, -0.31311899, -0.15495424],
            [ 0.01517388,  0.72535217,  0.38055357]])

array([[-1.10998738, -0.91063684,  1.18537974],
       [-1.11028981, -0.91011763,  1.18564749]], dtype=float32)

'ロード'

variable W([[ 0.60884207, -0.32222536, -0.14310044],
            [ 0.00407098,  0.71625102,  0.39241004]])

'重みはファイルに保存されていない'

array([[-1.10998738, -0.91063684,  1.18537974],
       [-1.11028981, -0.91011763,  1.18564749]], dtype=float32)